<a href="https://colab.research.google.com/github/ilovesylus/computer/blob/main/EX04_06_ET_Today_%E6%96%B0%E8%81%9E%E6%A8%99%E9%A1%8C%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 問題定義

爬取程式：ml_crawler_ettoday_news.ipynb
爬取過去一年 ET Today 的新聞標題做為資料集。
url = 'https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-ettoday_news.csv'
使用 Jieba 進行斷詞。
使用單純貝氏演算法進行新聞標題分類。

## 資料收集

In [2]:
import pandas as pd
url = 'https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-ettoday_news.csv'
df = pd.read_csv(url)
df.head()

,類別,標題
0,政治,青年座談不被AI取代的關鍵力 葉丙成：只有這兩種人才能真正把握時間紅利
1,政治,蕭美琴直播初體驗曝「咪琴嚴選」 邀網友挺花蓮
2,政治,疑似國軍「在美受訓」照曝光！ 陸軍不評論
3,政治,「蔡英文、柯文哲有談NCC」 府：沒談人選、政黨比例更沒私下喬
4,政治,總統府否認蔡英文向柯喬NCC委員 黃國昌嗆：醜聞被揭露還敢說謊


## 資料前清理



## 資料清理


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29216 entries, 0 to 29215
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   類別      29216 non-null  object
 1   標題      29216 non-null  object
dtypes: object(2)
memory usage: 456.6+ KB


### 探索性分析

In [9]:
!wget -O dict.txt https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-dict.txt
!wget -O stopwords.txt https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-stopwords.txt


--2025-06-15 07:00:32--  https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-dict.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4154480 (4.0M) [application/octet-stream]
Saving to: ‘dict.txt’

dict.txt            100%[===================>]   3.96M  --.-KB/s    in 0.02s   

2025-06-15 07:00:33 (216 MB/s) - ‘dict.txt’ saved [4154480/4154480]

--2025-06-15 07:00:33--  https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8129 (7.9K

In [11]:
import jieba
jieba.set_dictionary('dict.txt')
def load_stopwords(filepath):
  with open (filepath, 'r', encoding='utf-8') as f:
    stopwords = set(line.strip() for line in f if line.strip())
  return stopwords

def jieba_cut_with_stopwords(text, stopwords):
  words = jieba.cut(text)
  return [word for word in words  if word not in stopwords and word.strip() !='']
stopwords = load_stopwords('stopwords.txt')

In [13]:
df['標題'] = df['標題'].apply (lambda x: ''.join(jieba_cut_with_stopwords(x,stopwords)))


Building prefix dict from /content/dict.txt ...
DEBUG:jieba:Building prefix dict from /content/dict.txt ...
Dumping model to file cache /tmp/jieba.ueb620ec8402181953a0299d7957c0d6e.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.ueb620ec8402181953a0299d7957c0d6e.cache
Loading model cost 1.193 seconds.
DEBUG:jieba:Loading model cost 1.193 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [14]:
df

,類別,標題
0,政治,青年座談AI取代關鍵力葉丙成這兩種人才真正把握時間紅利
1,政治,蕭美琴直播體驗曝咪琴嚴選邀網友挺花蓮
2,政治,疑似國軍美受訓曝光陸軍評論
3,政治,蔡英文柯文哲有談NCC府沒談人選政黨比例私下喬
4,政治,總統府否認蔡英文柯喬NCC委員黃國昌嗆醜聞揭露還敢說謊
...,...,...
29211,生活,瞞未婚夫下海拍片休息月後I級AV女優宣布引退原因曝
29212,生活,努力考上清大家人句嫁有錢工程師心冷
29213,生活,北榮藝廊閃一週陳威明鎮館寶古柏園象徵醫者精神
29214,生活,收到一整串粽子媽氣炸大票愣每年吃到吐沒聽過


In [15]:
X,y = df['標題'], df['類別']

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### 資料分割

### 類別轉換

In [17]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

### 特徵縮放

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


## 模型訓練

In [19]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB(alpha = 1.0)
mnb.fit(X_train,y_train)


MultinomialNB()

In [20]:
mnb.score(X_test, y_test)

0.13535249828884327

## 模型評估

In [21]:
from sklearn.metrics import accuracy_score
y_pred = mnb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.13535249828884327


## 模型調整

## 模型部署

### 模型評估

In [22]:
from sklearn.metrics import accuracy_score
y_pred = mnb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.13535249828884327


### 推論預測

In [23]:
text = "桌球/開戰? 嗆體育署憑一面之詞霸凌桌協 選訓委員:誤用政治手段處理"
s = vectorizer.transform([''.join(jieba_cut_with_stopwords(text,stopwords))])
c = mnb.predict(s)[0]
category = label_encoder.classes_[c]
print(category)

財經
